# https://nlp100.github.io/ja/ch04.html

## 第5章: 係り受け解析  
日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

ステップ1: 依存関係のインストール  
sudo apt update  
sudo apt install -y build-essential curl file git  
sudo apt install -y mecab libmecab-dev mecab-ipadic-utf8  
sudo apt install -y zlib1g-dev libcurl4-openssl-dev libxml2-dev  

ステップ2: CRF++のインストール  
git clone https://github.com/taku910/crfpp.git  
cd crfpp  
./configure  
make  
sudo make install  
cd ..  

ステップ3: CaboChaのダウンロードとビルド  
git clone https://github.com/taku910/cabocha.git  
cd cabocha  
sudo apt-get install autoconf automake libtool  
autoreconf -i  
./configure --with-mecab-config=`which mecab-config` --with-charset=utf8  
make  
sudo make install  

swig_import_helper`について、以下の部分を修正
```
def swig_import_helper():
    import os  
    import importlib.util  
    spec = importlib.util.find_spec('_CaboCha', [os.path.dirname(__file__)])  
    if spec is None:  
        raise ImportError('_CaboCha module not found')  
    _mod = importlib.util.module_from_spec(spec)  
    spec.loader.exec_module(_mod)  
    return _mod  ```

In [2]:
import CaboCha
CBC = CaboCha.Parser()
divtext = []
with open("./datafiles/ai.ja.txt", "r") as f, open("./datafiles/ai.ja.txt.parsed", "w") as f2:
    lines = f.readlines()
    for text in lines:
        if "。" in text:
            temp = text.split("。")
            temp = [x + "。" for x in temp if x != '']
            divtext.extend(temp)
    for text in divtext:
        tree = CBC.parse(text)
        f2.write(tree.toString(CaboCha.FORMAT_LATTICE))

iconv_open is not supported


In [8]:
!head -5 ./datafiles/ai.ja.txt

人工知能

人工知能（じんこうちのう、、AI〈エーアイ〉）とは、「『計算（）』という概念と『コンピュータ（）』という道具を用いて『知能』を研究する計算機科学（）の一分野」を指す語。「言語の理解や推論、問題解決などの知的行動を人間に代わってコンピューターに行わせる技術」、または、「計算機（コンピュータ）による知的な情報処理システムの設計や実現に関する研究分野」ともされる。

『日本大百科全書(ニッポニカ)』の解説で、情報工学者・通信工学者の佐藤理史は次のように述べている。


In [6]:
!head -20 ./datafiles/ai.ja.txt.parsed

* 0 14D 1/1 -1.776924
人工	名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
知能	名詞,一般,*,*,*,*,知能,チノウ,チノー
* 1 14D 2/3 -1.776924
（	記号,括弧開,*,*,*,*,（,（,（
じん	名詞,一般,*,*,*,*,じん,ジン,ジン
こうち	名詞,一般,*,*,*,*,こうち,コウチ,コーチ
のう	助詞,終助詞,*,*,*,*,のう,ノウ,ノー
、	記号,読点,*,*,*,*,、,、,、
、	記号,読点,*,*,*,*,、,、,、
* 2 3D 0/0 0.592011
AI	名詞,一般,*,*,*,*,*
* 3 14D 1/5 -1.776924
〈	記号,括弧開,*,*,*,*,〈,〈,〈
エーアイ	名詞,固有名詞,一般,*,*,*,*
〉	記号,括弧閉,*,*,*,*,〉,〉,〉
）	記号,括弧閉,*,*,*,*,）,）,）
と	助詞,格助詞,引用,*,*,*,と,ト,ト
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
、	記号,読点,*,*,*,*,、,、,、


## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ

In [22]:
class Morph:
    def __init__(self, pos):
        """
        形態素を初期化します。
        :param pos: 形態素情報が格納されたリスト
        """
        self.surface = pos[0]  # 表層形
        self.base = pos[7]     # 基本形
        self.pos = pos[1]      # 品詞
        self.pos1 = pos[2]     # 品詞細分類1

    def __repr__(self):
        return f"Morph(surface='{self.surface}', base='{self.base}', pos='{self.pos}', pos1='{self.pos1}')"

    @classmethod # Morphクラスから直接メソッドを呼び出すために指定
    def parse_from_file(cls, file_path):
        """
        形態素解析結果を読み込み、文ごとの Morph オブジェクトのリストを生成します。
        :param file_path: ファイルのパス
        :return: 文ごとの Morph オブジェクトのリスト
        """
        morph_list = []
        sentence = []

        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()  # 空白や改行を除去
                if line == "EOS":  # 文の区切り
                    if sentence:
                        morph_list.append(sentence)
                        sentence = []
                elif not line.startswith("*"):  # 係り受け情報をスキップ
                    cols = line.split("\t")
                    if len(cols) > 1:
                        features = cols[1].split(",")
                        pos = [cols[0]] + features
                        sentence.append(cls(pos))  # クラスメソッドを使ってインスタンスを生成

        return morph_list


# ファイルパスを指定して実行
file_path = "./datafiles/ai.ja.txt.parsed"
morph_list = Morph.parse_from_file(file_path)

# 冒頭の文の形態素列を表示
if morph_list:
    print("冒頭の文の形態素列:")
    for morph in morph_list[0]:
        print(morph)

冒頭の文の形態素列:
Morph(surface='人工', base='人工', pos='名詞', pos1='一般')
Morph(surface='知能', base='知能', pos='名詞', pos1='一般')
Morph(surface='（', base='（', pos='記号', pos1='括弧開')
Morph(surface='じん', base='じん', pos='名詞', pos1='一般')
Morph(surface='こうち', base='こうち', pos='名詞', pos1='一般')
Morph(surface='のう', base='のう', pos='助詞', pos1='終助詞')
Morph(surface='、', base='、', pos='記号', pos1='読点')
Morph(surface='、', base='、', pos='記号', pos1='読点')
Morph(surface='AI', base='*', pos='名詞', pos1='一般')
Morph(surface='〈', base='〈', pos='記号', pos1='括弧開')
Morph(surface='エーアイ', base='*', pos='名詞', pos1='固有名詞')
Morph(surface='〉', base='〉', pos='記号', pos1='括弧閉')
Morph(surface='）', base='）', pos='記号', pos1='括弧閉')
Morph(surface='と', base='と', pos='助詞', pos1='格助詞')
Morph(surface='は', base='は', pos='助詞', pos1='係助詞')
Morph(surface='、', base='、', pos='記号', pos1='読点')
Morph(surface='「', base='「', pos='記号', pos1='括弧開')
Morph(surface='『', base='『', pos='記号', pos1='括弧開')
Morph(surface='計算', base='計算', pos='名詞', pos1='サ変接続')
Morph(su

## 41. 係り受け解析結果の読み込み（文節・係り受け）  
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ

In [25]:
class Chunk:
    def __init__(self, morphs, dst):
        """
        文節を表すクラス
        :param morphs: Morphオブジェクトのリスト
        :param dst: 係り先文節インデックス番号
        """
        self.morphs = morphs  # 形態素（Morphオブジェクト）のリスト
        self.dst = dst        # 係り先文節インデックス番号
        self.srcs = []        # 係り元文節インデックス番号のリスト

    def __repr__(self):
        morph_text = ''.join([morph.surface for morph in self.morphs if morph.pos != "記号"])
        return f"Chunk(text='{morph_text}', dst={self.dst}, srcs={self.srcs})"


def parse_chunks(file_path):
    """
    係り受け解析結果を読み込み、文ごとの Chunk オブジェクトのリストを生成する。
    :param file_path: ファイルのパス
    :return: 文ごとの Chunk オブジェクトのリスト
    """
    sentences = []
    chunks = []
    morphs = []
    dst = -1

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line == "EOS":
                if chunks:
                    for i, chunk in enumerate(chunks):
                        # dstが有効な範囲内であることを確認
                        if 0 <= chunk.dst < len(chunks):
                            chunks[chunk.dst].srcs.append(i)
                    sentences.append(chunks)
                chunks = []
            elif line.startswith("*"):
                if morphs:
                    chunks.append(Chunk(morphs, dst))
                    morphs = []
                _, idx, dst, *_ = line.split()
                dst = int(dst.rstrip("D"))
            else:
                cols = line.split("\t")
                if len(cols) > 1:
                    features = cols[1].split(",")
                    pos = [cols[0]] + features
                    morphs.append(Morph(pos))
        if morphs:
            chunks.append(Chunk(morphs, dst))

    return sentences


# ファイルパスを指定して実行
file_path = "./datafiles/ai.ja.txt.parsed"
chunk_list = parse_chunks(file_path)

# 冒頭の文の文節と係り先を表示
if chunk_list:
    print("冒頭の文の文節と係り先:")
    for i, chunk in enumerate(chunk_list[0]):
        # 文節の文字列を取得
        chunk_text = ''.join([m.surface for m in chunk.morphs if m.pos != "記号"])
        
        # 係り先の文字列を取得（範囲チェック付き）
        if 0 <= chunk.dst < len(chunk_list[0]):
            dst_text = ''.join([m.surface for m in chunk_list[0][chunk.dst].morphs if m.pos != "記号"])
        else:
            dst_text = "なし"
        
        print(f"文節 {i}: {chunk_text} -> {dst_text}")

冒頭の文の文節と係り先:
文節 0: 人工知能 -> なし
文節 1: じんこうちのう -> なし
文節 2: AI -> エーアイとは
文節 3: エーアイとは -> なし
文節 4: 計算という -> コンピュータという
文節 5: 概念と -> コンピュータという
文節 6: コンピュータという -> 道具を
文節 7: 道具を -> 用いて
文節 8: 用いて -> 研究する
文節 9: 知能を -> 研究する
文節 10: 研究する -> 計算機科学の
文節 11: 計算機科学の -> 一分野を
文節 12: 一分野を -> 指す
文節 13: 指す -> なし


## 42. 係り元と係り先の文節の表示  
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [33]:
class Chunk:
    def __init__(self, morphs, dst):
        """
        文節を表すクラス
        :param morphs: Morphオブジェクトのリスト
        :param dst: 係り先文節インデックス番号
        """
        self.morphs = morphs  # 形態素（Morphオブジェクト）のリスト
        self.dst = dst        # 係り先文節インデックス番号
        self.srcs = []        # 係り元文節インデックス番号のリスト

    def __repr__(self):
        morph_text = ''.join([morph.surface for morph in self.morphs if morph.pos != "記号"])
        return f"Chunk(text='{morph_text}', dst={self.dst}, srcs={self.srcs})"


def parse_chunks(file_path):
    """
    係り受け解析結果を読み込み、文ごとの Chunk オブジェクトのリストを生成する。
    :param file_path: ファイルのパス
    :return: 文ごとの Chunk オブジェクトのリスト
    """
    sentences = []
    chunks = []
    morphs = []
    dst = -1

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line == "EOS":
                if chunks:
                    for i, chunk in enumerate(chunks):
                        # dstが有効な範囲内であることを確認
                        if 0 <= chunk.dst < len(chunks):
                            chunks[chunk.dst].srcs.append(i)
                    sentences.append(chunks)
                chunks = []
            elif line.startswith("*"):
                if morphs:
                    chunks.append(Chunk(morphs, dst))
                    morphs = []
                _, idx, dst, *_ = line.split()
                dst = int(dst.rstrip("D"))
            else:
                cols = line.split("\t")
                if len(cols) > 1:
                    features = cols[1].split(",")
                    pos = [cols[0]] + features
                    morphs.append(Morph(pos))
        if morphs:
            chunks.append(Chunk(morphs, dst))

    return sentences


# ファイルパスを指定して実行
file_path = "./datafiles/ai.ja.txt.parsed"
chunk_list = parse_chunks(file_path)

# 冒頭の文の文節と係り先を表示
if chunk_list:
    print("冒頭3文の文節と係り先:")
    for sentence_idx, sentence in enumerate(chunk_list[:3]):  # 冒頭3文
        print(f"\n【文 {sentence_idx + 1}】")
        for i, chunk in enumerate(sentence):
            # 文節の文字列を取得
            chunk_text = ''.join([m.surface for m in chunk.morphs if m.pos != "記号"])
            
            # 係り先の文字列を取得（範囲チェック付き）
            if 0 <= chunk.dst < len(sentence):
                dst_text = ''.join([m.surface for m in sentence[chunk.dst].morphs if m.pos != "記号"])
            else:
                dst_text = "なし"
            
            print(f"文節 {i}: {chunk_text}\t{dst_text}")

冒頭3文の文節と係り先:

【文 1】
文節 0: 人工知能	なし
文節 1: じんこうちのう	なし
文節 2: AI	エーアイとは
文節 3: エーアイとは	なし
文節 4: 計算という	コンピュータという
文節 5: 概念と	コンピュータという
文節 6: コンピュータという	道具を
文節 7: 道具を	用いて
文節 8: 用いて	研究する
文節 9: 知能を	研究する
文節 10: 研究する	計算機科学の
文節 11: 計算機科学の	一分野を
文節 12: 一分野を	指す
文節 13: 指す	なし

【文 2】
文節 0: 語	なし
文節 1: 言語の	理解や
文節 2: 理解や	理解や
文節 3: 推論	推論
文節 4: 問題解決などの	問題解決などの
文節 5: 知的行動を	人間に
文節 6: 人間に	人間に
文節 7: 代わって	コンピューターに
文節 8: コンピューターに	コンピューターに
文節 9: 行わせる	行わせる
文節 10: 技術または	実現に関する
文節 11: 計算機コンピュータによる	知的な
文節 12: 知的な	知的な
文節 13: 情報処理システムの	設計や
文節 14: 設計や	設計や
文節 15: 実現に関する	実現に関する
文節 16: 研究分野とも	研究分野とも

【文 3】
文節 0: される	なし
